In [1]:
## Convert data to a excel file 
import pandas as pd
import glob
import json
import os

# Path to your JSON files directory
json_folder_path = "/Users/wesly.alves/Library/Mobile Documents/com~apple~CloudDocs/!Pessoal/Python/json"  # Update with your actual path

json_files = glob.glob(os.path.join(json_folder_path, "*.json"))

# Define the output Excel file name
output_file = "merged_data.xlsx"

# List to hold individual DataFrames for merging
dataframes = []

# Create an Excel writer object
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Loop through each JSON file
    for json_file in json_files:
        with open(json_file, 'r') as file:
            # Load JSON data
            data = json.load(file)

            # Check if the data is a dictionary (so we can use .get), otherwise use data as-is
            if isinstance(data, dict):
                data_to_df = data.get("data", data)  # Get 'data' or full content if no 'data' key
            else:
                data_to_df = data  # If it's a list or other structure, use data directly

            # Convert to DataFrame
            df = pd.json_normalize(data_to_df)  # Flatten JSON if necessary

            # Append to the list for merging
            dataframes.append(df)

            # Use the filename (without .json) as the sheet name
            sheet_name = os.path.basename(json_file).replace('.json', '')

            # Write the DataFrame to a separate sheet in the Excel file
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Combine all DataFrames, using `join='outer'` to handle different columns
    combined_df = pd.concat(dataframes, ignore_index=True, sort=False)

    # Write the combined DataFrame to a new sheet
    combined_df.to_excel(writer, sheet_name="All_Data_Combined", index=False)

print(f"Data successfully saved to {output_file}")


Data successfully saved to merged_data.xlsx


In [2]:
import pandas as pd
from datetime import datetime
import pytz

# Function to convert epoch time (in milliseconds) to human-readable format with timezone awareness
def convert_epoch_to_human(epoch_time, timezone='UTC'):
    epoch_time_seconds = epoch_time / 1000  # Convert milliseconds to seconds
    tz = pytz.timezone(timezone)
    dt = datetime.fromtimestamp(epoch_time_seconds, tz=pytz.utc).astimezone(tz)
    return dt.strftime('%m-%d-%Y') #%H:%M:%S'

# Read the Excel file with all sheets
df_sheets = pd.read_excel('merged_data.xlsx', sheet_name=None)  # 'sheet_name=None' reads all sheets into a dictionary

# Specify the desired timezone for conversion
timezone = 'UTC'  # Modify as needed, e.g., 'America/New_York'

# Keywords to look for in column names
keywords = ['created_at', 'modified_at', 'timestamp','last_scan_timestamp', 'next_scan_timestamp', 'timestamp', 'published_at']  # Add other keywords as needed

# Open an Excel writer to save all processed sheets in one output file
with pd.ExcelWriter('data_with_human_dates.xlsx', engine='openpyxl') as writer:
    # Loop through each sheet in the Excel file
    for sheet_name, sheet_data in df_sheets.items():
        # Identify columns that match any keyword in the list
        for column in sheet_data.columns:
            if any(keyword in column.lower() for keyword in keywords):  # Check if any keyword exists in the column name
                # Apply conversion only if the column is likely an epoch timestamp in milliseconds
                sheet_data[column] = sheet_data[column].apply(
                    lambda x: convert_epoch_to_human(x, timezone) if isinstance(x, (int, float)) and (x > 1000000000000) else x
                )
        
        # Write the processed sheet to the Excel file
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)

print("Epoch times converted and saved to 'data_with_human_dates.xlsx'")


Epoch times converted and saved to 'data_with_human_dates.xlsx'
